In [11]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [12]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [13]:

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [14]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

### Running in Docker

Run the following command from terminal:

```
    docker run -it \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

(-p: port forwarding ollama server 11434 - Docker container 11434)

Pulling the model into Docker container:

```    
    docker exec -it ollama bash
    ollama pull phi3
```

(-it: interactive mode)

### Connecting to Ollama with OpenAI API:

In [16]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [17]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
llm('write that this is a test')

' This is a test.\n\n\nTo perform the task, I took the phrase provided and reproduced it in its simplest form with no additional information or context added: "This is a test." It\'s important to note that without further instructions regarding format (e.g., capitalization), style of writing (formal/informal), audience targeted, etc., only this basic sentence structure has been used here for clarity and simplicity as requested by the instruction.'

In [19]:
print(_)

 This is a test.


To perform the task, I took the phrase provided and reproduced it in its simplest form with no additional information or context added: "This is a test." It's important to note that without further instructions regarding format (e.g., capitalization), style of writing (formal/informal), audience targeted, etc., only this basic sentence structure has been used here for clarity and simplicity as requested by the instruction.
